In [ ]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
data = pd.read_csv("/content/Spam Email raw text for NLP.csv", encoding='latin-1')

In [ ]:
data.head()

,CATEGORY,MESSAGE,FILE_NAME
0,1,"Dear Homeowner,\n\n \n\nInterest Rates are at ...",00249.5f45607c1bffe89f60ba1ec9f878039a
1,1,ATTENTION: This is a MUST for ALL Computer Use...,00373.ebe8670ac56b04125c25100a36ab0510
2,1,This is a multi-part message in MIME format.\n...,00214.1367039e50dc6b7adb0f2aa8aba83216
3,1,IMPORTANT INFORMATION:\n\n\n\nThe new domain n...,00210.050ffd105bd4e006771ee63cabc59978
4,1,This is the bottom line. If you can GIVE AWAY...,00033.9babb58d9298daa2963d4f514193d7d6


In [ ]:
data.drop('FILE_NAME', axis = 1, inplace = True)

In [ ]:
data

,CATEGORY,MESSAGE
0,1,"Dear Homeowner,\n\n \n\nInterest Rates are at ..."
1,1,ATTENTION: This is a MUST for ALL Computer Use...
2,1,This is a multi-part message in MIME format.\n...
3,1,IMPORTANT INFORMATION:\n\n\n\nThe new domain n...
4,1,This is the bottom line. If you can GIVE AWAY...
...,...,...
5791,0,"I'm one of the 30,000 but it's not working ver..."
5792,0,Damien Morton quoted:\n\n>W3C approves HTML 4 ...
5793,0,"On Mon, 2002-07-22 at 06:50, che wrote:\n\n\n\..."
5794,0,"Once upon a time, Manfred wrote :\n\n\n\n> I w..."


In [ ]:
data.CATEGORY.value_counts()

0    3900
1    1896
Name: CATEGORY, dtype: int64

In [ ]:
stopword = nltk.corpus.stopwords.words('english')

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
corpus = []

In [ ]:
for i in range(len(data)):
  # Remove all non-aplhanumeric characters
  messages = re.sub('[^a-zA-Z0-9]', ' ', data['MESSAGE'][i])

  # Convert to lower case
  messages = messages.lower()

  # Split into words for lemmetization
  messages = messages.split()

  # Remove stopwords and lemmatizing
  messages = [lemmatizer.lemmatize(word) for word in messages
              if word not in set(stopwords.words('english'))]

  # Convert words back to sentences
  messages = ' '.join(messages)

  # Adding preprocessed data into corpus
  corpus.append(messages)

In [ ]:
cv = CountVectorizer(max_features = 2500, ngram_range = (1,3))
X = cv.fit_transform(corpus).toarray()
y = data.CATEGORY

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 1, stratify = y)

In [ ]:
tf = TfidfVectorizer(ngram_range = (1,3), max_features = 2500)
X = tf.fit_transform(corpus).toarray()


In [ ]:
import pickle
pickle.dump(tf, open('cv.pkl', 'wb'))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 1, stratify = y)

In [ ]:
model = MultinomialNB()

In [ ]:
model.fit(X_train, y_train)

MultinomialNB()

In [ ]:
train_pred = model.predict(X_train)
test_pred = model.predict(X_test)

In [ ]:
print(classification_report(train_pred, y_train))
print(classification_report(test_pred, y_test))

              precision    recall  f1-score   support

           0       0.99      0.95      0.97      3258
           1       0.89      0.98      0.94      1378

    accuracy                           0.96      4636
   macro avg       0.94      0.97      0.95      4636
weighted avg       0.96      0.96      0.96      4636

              precision    recall  f1-score   support

           0       0.99      0.96      0.97       812
           1       0.91      0.99      0.94       348

    accuracy                           0.96      1160
   macro avg       0.95      0.97      0.96      1160
weighted avg       0.97      0.96      0.97      1160



In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score
cm = confusion_matrix(y_test, test_pred)
score = accuracy_score(y_test,test_pred)
print(cm,score*100)

[[776   5]
 [ 36 343]] 96.4655172413793


In [ ]:
pickle.dump(model, open("spam.pkl", "wb"))

In [ ]:
loaded_model = pickle.load(open("spam.pkl", "rb"))
loaded_model.predict(X_test)
loaded_model.score(X_test,y_test)

0.9646551724137931

In [ ]:
def new_review(new_review):
  new_review = new_review
  new_review = re.sub('[^a-zA-Z]', ' ', new_review)
  new_review = new_review.lower()
  new_review = new_review.split()
  lemmatizer = WordNetLemmatizer()
  new_review = [lemmatizer.lemmatize(word) for word in new_review
              if word not in set(stopwords.words('english'))]
  new_review = ' '.join(new_review)
  new_corpus = [new_review]
  new_X_test = cv.transform(new_corpus).toarray()
  new_X_test = tf.transform(new_corpus).toarray()
  new_y_pred = loaded_model.predict(new_X_test)

  return new_y_pred

new_review = new_review(str(input("Enter new review...")))
if new_review[0]==1:
  print("Spam")
else :
  print("Not spam")

Enter new review...You won 10000 dollars, please provide your account details,So that we can transfer the money
Spam


In [ ]:
print('Predicting...')

message = ["You won 10000 dollars, please provide your account details,So that we can transfer the money"]

message_vector = tf.transform(message)
category = model.predict(message_vector)
print("The message is", "spam" if category == 1 else "not spam")

Predicting...
The message is spam
